In [1]:
pip install flask-ngrok

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/YOLO Project/yolo11_flask_web

/content/drive/MyDrive/YOLO Project/yolo11_flask_web


In [4]:
ls

app.py  best.pt  Deploy_yolo.ipynb  templates/  uploads/  yolo11s.pt


In [5]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 923.8/923.8 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [6]:
from flask_ngrok import run_with_ngrok
import pickle
import numpy as np
import os
import cv2
import numpy as np
from flask import Flask, render_template, Response, request, redirect, url_for, send_from_directory
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
!pip install pyngrok

In [8]:
!ngrok authtoken 2muuEr3svya7o5cYWzx4JOSuDj2_4Hfh7Sg2b7ay2K21RCCS3

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


#Deploy best.pt on web application

In [10]:
import os  # ใช้สำหรับการทำงานกับระบบไฟล์
import cv2  # OpenCV สำหรับการจัดการภาพและวิดีโอ
import numpy as np  # ใช้สำหรับการคำนวณทางคณิตศาสตร์
from flask import Flask, render_template, Response, request, redirect, url_for, send_from_directory  # Flask สำหรับสร้าง Web Application
from ultralytics import YOLO  # YOLO สำหรับการโหลดโมเดล YOLO
from pyngrok import ngrok  # Pyngrok สำหรับสร้าง public URL เพื่อเข้าถึงแอปผ่านอินเทอร์เน็ต

app = Flask(__name__)  # สร้าง Flask application

run_with_ngrok(app)  # เรียกใช้ ngrok เพื่อให้แอปสามารถเข้าถึงได้ผ่าน public URL

# โหลดโมเดล YOLOv8
model = YOLO("best.pt")  # โหลดโมเดล YOLO จากไฟล์ 'best.pt'
names = model.model.names  # ดึงชื่อของคลาสจากโมเดล

ngrok_tunnel = ngrok.connect(5000)  # เปิดการเชื่อมต่อ ngrok บนพอร์ต 5000
print('Public URL:', ngrok_tunnel.public_url)  # พิมพ์ public URL ของ ngrok

@app.route('/')  # กำหนดเส้นทาง '/' (หน้าแรก)
def index():
    return render_template('index.html')  # แสดงหน้า HTML ที่ชื่อว่า 'index.html'

@app.route('/start_webcam')
def start_webcam():
    return redirect(url_for('index'))  # Redirect to homepage

def detect_objects_from_webcam():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (1020, 600))
        results = model.track(frame, persist=True)
        if results[0].boxes is not None and results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.int().cpu().tolist()
            class_ids = results[0].boxes.cls.int().cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            for box, class_id, track_id in zip(boxes, class_ids, track_ids):
                c = names[class_id]
                x1, y1, x2, y2 = box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'{track_id} - {c}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)
        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/webcam_feed')
def webcam_feed():
    return Response(detect_objects_from_webcam(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/upload', methods=['POST'])  # เส้นทางสำหรับอัปโหลดวิดีโอ
def upload_video():
    if 'file' not in request.files:  # ตรวจสอบว่ามีไฟล์ในคำขอหรือไม่
        return redirect(request.url)

    file = request.files['file']  # รับไฟล์จากคำขอ
    if file.filename == '':
        return redirect(request.url)  # ถ้าไม่มีชื่อไฟล์ให้กลับไปหน้าเดิม

    if not os.path.exists('uploads'):  # ตรวจสอบว่ามีโฟลเดอร์ 'uploads' หรือไม่
        os.makedirs('uploads')  # ถ้าไม่มีให้สร้างขึ้น

    file_path = os.path.join('uploads', file.filename)  # สร้างพาธไฟล์
    file.save(file_path)  # บันทึกไฟล์ลงโฟลเดอร์

    return redirect(url_for('play_video', filename=file.filename))  # เปลี่ยนเส้นทางไปยังหน้าเล่นวิดีโอ

@app.route('/uploads/<filename>')  # เส้นทางสำหรับเล่นวิดีโอ
def play_video(filename):
    return render_template('play_video.html', filename=filename)  # แสดงหน้า HTML พร้อมเล่นวิดีโอ

@app.route('/video/<path:filename>')  # เส้นทางสำหรับให้ดาวน์โหลดวิดีโอ
def send_video(filename):
    return send_from_directory('uploads', filename)  # ส่งไฟล์วิดีโอจากโฟลเดอร์ 'uploads'

# ฟังก์ชันตรวจจับวัตถุจากวิดีโอที่อัปโหลด
def detect_objects_from_video(video_path):
    cap = cv2.VideoCapture(video_path)  # เปิดไฟล์วิดีโอ
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()  # อ่านเฟรมจากวิดีโอ
        if not ret:
            break  # หยุดถ้าไม่มีเฟรมใหม่
        count += 1
        if count % 2 != 0:
           continue  # ข้ามเฟรมที่นับเป็นเลขคี่

        frame = cv2.resize(frame, (1020, 600))  # ปรับขนาดเฟรม

        results = model.track(frame, persist=True)  # ตรวจจับและติดตามวัตถุ

        if results[0].boxes is not None and results[0].boxes.id is not None:  # ตรวจสอบว่ามีการตรวจจับ
            boxes = results[0].boxes.xyxy.int().cpu().tolist()
            class_ids = results[0].boxes.cls.int().cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            for box, class_id, track_id in zip(boxes, class_ids, track_ids):  # วนลูปในวัตถุที่ตรวจจับ
                c = names[class_id]
                x1, y1, x2, y2 = box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # วาดกรอบ
                cv2.putText(frame, f'{track_id} - {c}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1)

        _, buffer = cv2.imencode('.jpg', frame)  # เข้ารหัสเฟรมเป็น JPEG
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/video_feed/<filename>')  # เส้นทางสำหรับแสดงผลวิดีโอจากไฟล์อัปโหลด
def video_feed(filename):
    video_path = os.path.join('uploads', filename)  # สร้างพาธของไฟล์วิดีโอ
    return Response(detect_objects_from_video(video_path),
                    mimetype='multipart/x-mixed-replace; boundary=frame')  # ส่งเฟรมแบบไลฟ์สตรีม

app.run()  # รันแอปพลิเคชัน Flask


Public URL: https://1413-34-125-27-23.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://1413-34-125-27-23.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 13:58:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 13:58:59] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 14:00:05] "POST /upload HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 14:00:06] "GET /uploads/Broiler_chicken.mp4 HTTP/1.1" 200 -


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.8 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 2.2s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 384x640 18 chickens, 53.9ms
Speed: 11.2ms preprocess, 53.9ms inference, 336.4ms postprocess per image at shape (1, 3, 384, 640)


INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 14:00:14] "GET /video_feed/Broiler_chicken.mp4 HTTP/1.1" 200 -



0: 384x640 18 chickens, 10.0ms
Speed: 2.7ms preprocess, 10.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 chickens, 9.3ms
Speed: 2.6ms preprocess, 9.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 20 chickens, 17.5ms
Speed: 3.0ms preprocess, 17.5ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 chickens, 16.4ms
Speed: 3.1ms preprocess, 16.4ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 24 chickens, 12.6ms
Speed: 3.0ms preprocess, 12.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 chickens, 16.1ms
Speed: 4.2ms preprocess, 16.1ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 chickens, 18.0ms
Speed: 3.1ms preprocess, 18.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 25 chickens, 11.9ms
Speed: 2.9ms preprocess, 11.9ms inference, 1.8ms postprocess per im

INFO:werkzeug:127.0.0.1 - - [11/Mar/2025 14:02:40] "GET / HTTP/1.1" 200 -



0: 384x640 32 chickens, 9.3ms
Speed: 3.8ms preprocess, 9.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 30 chickens, 9.1ms
Speed: 3.1ms preprocess, 9.1ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 chickens, 9.7ms
Speed: 2.8ms preprocess, 9.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 chickens, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 chickens, 13.3ms
Speed: 3.1ms preprocess, 13.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 31 chickens, 9.4ms
Speed: 3.0ms preprocess, 9.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 29 chickens, 9.1ms
Speed: 2.8ms preprocess, 9.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 30 chickens, 13.9ms
Speed: 3.1ms preprocess, 13.9ms inference, 1.4ms postprocess per image at sha